## Colour Map API Examples

This notebook demonstrates how to work with Evo Colour Maps using **direct API calls**, providing granular control and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct Colour Map API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `ColormapAPIClient`
- Automatic handling of complex API interactions
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

### Display all objects in the workspace

Pick an object using dropdowns grouped by schema type. This uses direct API calls to fetch and organize workspace objects.

In [ ]:
# Interactive object picker using raw API
import json
from collections import defaultdict
from http import HTTPStatus

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

from evo.common.data import HTTPResponse

# Fetch all objects via direct API
try:
    resource_path = "/geoscience-object/orgs/{org_id}/workspaces/{workspace_id}/objects"
    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list objects. Status: {status}")

    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)
    all_objects = response_json.get("objects", [])

    if len(all_objects) == 0:
        print("No objects found.")
    else:
        # Determine object type from schema, extracting the part after "/objects/" and before the next "/"
        def object_type(obj):
            schema = obj.get("schema") or ""
            if schema and "/objects/" in schema:
                after_objects = schema.split("/objects/")[-1]
                return after_objects.split("/")[0]
            return schema if schema else "Unknown"

        grouped = defaultdict(list)
        for obj in all_objects:
            grouped[object_type(obj)].append(obj)

        for objs in grouped.values():
            objs.sort(key=lambda o: o.get("name") or "")

        type_options = sorted(grouped.keys())

        def build_object_options(selected_type):
            # Dropdown expects (label, value) pairs
            return [
                ((o.get("name", "").removesuffix(".json")), str(o.get("object_id"))) for o in grouped[selected_type]
            ]

        # Dynamic widths based on content lengths
        def width_for_labels(labels, min_px=220, max_px=720, char_px=8):
            longest = max((len(lbl) for lbl in labels), default=0)
            return f"{min(max(min_px, longest * char_px + 40), max_px)}px"

        type_width = width_for_labels(type_options)
        object_width = width_for_labels([name for name, _ in build_object_options(type_options[0])])
        label_width = "110px"
        row_gap = "6px"

        type_dropdown = widgets.Dropdown(
            options=type_options,
            description="",
            disabled=False,
            layout=widgets.Layout(width=type_width),
        )

        objects_dropdown = widgets.Dropdown(
            options=build_object_options(type_options[0]),
            description="",
            disabled=False,
            layout=widgets.Layout(width=object_width),
        )

        def on_type_change(change):
            if change.get("name") == "value" and change.get("new") in grouped:
                new_options = build_object_options(change["new"])
                objects_dropdown.options = new_options
                objects_dropdown.layout.width = width_for_labels([name for name, _ in new_options])

        type_dropdown.observe(on_type_change, names="value")

        label_layout = Layout(width=label_width, display="flex", justify_content="flex-start", align_items="center")
        row_layout = Layout(align_items="center", gap=row_gap)
        type_row = widgets.HBox([widgets.Label("Object type", layout=label_layout), type_dropdown], layout=row_layout)
        objects_row = widgets.HBox([widgets.Label("Object", layout=label_layout), objects_dropdown], layout=row_layout)

        print(f"Found {len(all_objects)} object(s) across {len(type_options)} type(s). Select a type, then an object:")
        display(widgets.VBox([type_row, objects_row]))

        # Expose the selected value for downstream cells
        selected_object_dropdown = objects_dropdown
except Exception as e:
    print(f"Error building object picker:\n{e}")

### Select an attribute from the chosen object

Extract all attributes from the object and display them in a dropdown.

In [ ]:
import json
from http import HTTPStatus

import ipywidgets as widgets
from IPython.display import display

from evo.common.data import HTTPResponse


def extract_attributes(data, attributes=None):
    """Recursively extract all attributes with 'key' and 'name' properties."""
    if attributes is None:
        attributes = []

    if isinstance(data, dict):
        # Check if this dict has both 'key' and 'name' properties
        if "key" in data and "name" in data:
            attributes.append({"key": data["key"], "name": data["name"]})

        # Recursively search all values in the dict
        for value in data.values():
            extract_attributes(value, attributes)

    elif isinstance(data, list):
        # Recursively search all items in the list
        for item in data:
            extract_attributes(item, attributes)

    return attributes


# Fetch the selected object details to extract attributes
selected_object_id = selected_object_dropdown.value

try:
    obj_path = "/geoscience-object/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}"
    obj_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "object_id": selected_object_id,
    }
    obj_resp = await connector.call_api(
        method="GET",
        resource_path=obj_path,
        path_params=obj_params,
        response_types_map={"200": HTTPResponse},
    )

    if obj_resp.status != HTTPStatus.OK:
        print(f"Error: Failed to fetch object. Status: {obj_resp.status}")
    else:
        obj_json = json.loads(obj_resp.data.decode("utf-8"))

        # Extract all attributes recursively
        all_attributes = extract_attributes(obj_json)

        if not all_attributes:
            print("No attributes found in this object.")
        else:
            # Sort attributes alphabetically by name
            attribute_options = sorted([(attr["name"], attr["key"]) for attr in all_attributes], key=lambda x: x[0])

            # Create a dropdown widget for attribute selection
            attribute_dropdown = widgets.Dropdown(
                options=attribute_options,
                description="Attribute:",
                disabled=False,
                style={"description_width": "110px"},
                layout=widgets.Layout(width="500px"),
            )

            print(f"Found {len(all_attributes)} attribute(s) in the selected object:")
            display(attribute_dropdown)

except Exception as e:
    print(f"Error extracting attributes:\n{e}")

### Create and publish a category colormap

Create a new category colormap and associate it with the selected object attribute using direct API calls.

In [ ]:
import json
from http import HTTPStatus

from evo.common.data import HTTPResponse

# Get the selected attribute ID from the dropdown
selected_attribute_id = attribute_dropdown.value

# Define the category colormap with sample categories and colors
categories = ["Rock Type A", "Rock Type B", "Rock Type C", "Rock Type D"]
colors = [[255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0]]  # RGB arrays

# Step 1: Create the colormap via POST request
colormap_data = {
    "colors": colors,
    "map": categories,
    "name": f"Category colormap for {selected_object_id}",
    "dtype": "category",
}

try:
    create_cm_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/colormaps"
    create_cm_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    create_cm_resp = await connector.call_api(
        method="POST",
        resource_path=create_cm_path,
        path_params=create_cm_params,
        header_params={"Content-Type": "application/json", "Accept": "application/json"},
        body=colormap_data,
        response_types_map={"201": HTTPResponse},
    )

    if create_cm_resp.status != HTTPStatus.CREATED:
        print(f"Error: Failed to create colormap. Status: {create_cm_resp.status}")
    else:
        created_colormap = json.loads(create_cm_resp.data.decode("utf-8"))
        colormap_id = created_colormap.get("id")

        print("Colormap created successfully!")
        print(f"Colormap ID: {colormap_id}\n")

        # Step 2: Associate the colormap with the object attribute via POST request
        association_data = {
            "attribute_id": selected_attribute_id,
            "colormap_id": colormap_id,
        }

        assoc_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}/associations"
        assoc_params = {
            "org_id": org_id,
            "workspace_id": workspace_id,
            "object_id": selected_object_id,
        }

        assoc_resp = await connector.call_api(
            method="POST",
            resource_path=assoc_path,
            path_params=assoc_params,
            header_params={"Content-Type": "application/json", "Accept": "application/json"},
            body=association_data,
            response_types_map={"200": HTTPResponse},
        )

        if assoc_resp.status != HTTPStatus.OK:
            print(f"Error: Failed to create association. Status: {assoc_resp.status}")
        else:
            association = json.loads(assoc_resp.data.decode("utf-8"))
            association_id = association.get("id")

            print("Colormap associated with object attribute!")
            print(f"Association ID: {association_id}")

except Exception as e:
    print(f"Error creating colormap:\n{e}")

### Fetch colormap data and visualise

In [ ]:
# Fetch associations, colormaps, and visualise by type
import json
from datetime import datetime
from http import HTTPStatus
from uuid import UUID

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse


def serialize_for_display(obj):
    if isinstance(obj, UUID):
        return str(obj)
    elif isinstance(obj, datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_for_display(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_for_display(value) for key, value in obj.items()}
    else:
        return obj


# Download full object to cross-reference attribute names
async def get_attribute_name_for_id(object_id, attribute_id, org_id, workspace_id, connector):
    """Recursively find an attribute name by its ID in the object JSON."""

    def find_attribute_by_key(data, target_key):
        """Recursively search through data structure to find an item with matching key."""
        if isinstance(data, dict):
            # Check if this dict has a 'key' property matching our target
            if "key" in data and str(data.get("key")) == str(target_key):
                return data.get("name")
            # Recursively search all values in the dict
            for value in data.values():
                result = find_attribute_by_key(value, target_key)
                if result is not None:
                    return result
        elif isinstance(data, list):
            # Recursively search all items in the list
            for item in data:
                result = find_attribute_by_key(item, target_key)
                if result is not None:
                    return result
        return None

    try:
        obj_path = "/geoscience-object/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}"
        obj_params = {
            "org_id": org_id,
            "workspace_id": workspace_id,
            "object_id": object_id,
        }
        obj_resp = await connector.call_api(
            method="GET",
            resource_path=obj_path,
            path_params=obj_params,
            response_types_map={"200": HTTPResponse},
        )
        if obj_resp.status != HTTPStatus.OK:
            return None
        obj_json = json.loads(obj_resp.data.decode("utf-8"))

        # Use recursive search to find the attribute name
        attribute_name = find_attribute_by_key(obj_json, attribute_id)
        return attribute_name
    except Exception:
        return None


async def fetch_and_visualize_colormaps(object_id, org_id, workspace_id, connector):
    """Fetch colormap associations and visualize them."""
    try:
        assoc_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}/associations"
        assoc_params = {
            "org_id": org_id,
            "workspace_id": workspace_id,
            "object_id": object_id,
        }

        assoc_resp = await connector.call_api(
            method="GET",
            resource_path=assoc_path,
            path_params=assoc_params,
            response_types_map={"200": HTTPResponse},
        )

        if assoc_resp.status != HTTPStatus.OK:
            raise RuntimeError(f"Error: Failed to get associations. Status: {assoc_resp.status}")

        assoc_json = json.loads(assoc_resp.data.decode("utf-8"))
        associations = assoc_json.get("associations", []) or assoc_json.get("items", [])

        print("\nColormap associations for this object:\n")
        print(highlight(json.dumps(assoc_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

        for association in associations:
            colormap_id = association.get("colormap_id")
            attribute_id = association.get("attribute_id")
            attribute_name = await get_attribute_name_for_id(object_id, attribute_id, org_id, workspace_id, connector)

            print("\n" + "=" * 80)
            print(f"Colormap ID: {colormap_id}")
            print("=" * 80 + "\n")

            # Fetch colormap metadata
            cm_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/colormaps/{colormap_id}"
            cm_params = {
                "org_id": org_id,
                "workspace_id": workspace_id,
                "colormap_id": colormap_id,
            }
            cm_resp = await connector.call_api(
                method="GET",
                resource_path=cm_path,
                path_params=cm_params,
                response_types_map={"200": HTTPResponse},
            )

            if cm_resp.status != HTTPStatus.OK:
                print(f"Failed to fetch colormap {colormap_id}: {cm_resp.status}")
                continue

            cm_json = json.loads(cm_resp.data.decode("utf-8"))
            attribute_controls = cm_json.get("attribute_controls") or []
            gradient_controls = cm_json.get("gradient_controls") or []
            colors = cm_json.get("colors") or []
            category_map = cm_json.get("map") or []

            # Determine type heuristically based on available fields
            if category_map:
                colormap_type = "CategoryColormap"
            elif attribute_controls:
                colormap_type = "ContinuousColormap"
            else:
                colormap_type = "DiscreteColormap"

            print(f"{colormap_type} for attribute: {attribute_name or 'Unknown'}")

            # Normalize colors to 0-1 for matplotlib
            normalized_colors = [tuple(c / 255.0 for c in color) for color in colors]

            if colormap_type == "CategoryColormap":
                fig, ax = plt.subplots(figsize=(10, max(1, len(colors) * 0.5)))
                ax.set_xlim(0, 1)
                ax.set_ylim(0, len(colors))
                ax.axis("off")

                for i, (label, color) in enumerate(zip(category_map, normalized_colors)):
                    y_pos = len(colors) - i - 1
                    rect = mpatches.Rectangle((0, y_pos), 0.15, 0.8, facecolor=color, edgecolor="black", linewidth=0.5)
                    ax.add_patch(rect)
                    ax.text(0.18, y_pos + 0.4, f"{label}", va="center", fontsize=10)
                    print(f"{label}: {colors[i]}")

                plt.title(
                    f"Category colormap for attribute: {attribute_name or 'Unknown'}", fontsize=12, pad=10, loc="left"
                )
                plt.tight_layout()
                plt.show()

            elif colormap_type == "DiscreteColormap":
                end_points = cm_json.get("end_points") or []
                end_inclusive = cm_json.get("end_inclusive") or []

                print("Colors:")
                for color in colors:
                    print(f"{color}")

                print("End points:", end_points)
                print("End inclusive:", end_inclusive)

            elif colormap_type == "ContinuousColormap":
                print(f"Attribute controls: {attribute_controls}")
                print(f"Gradient controls: {gradient_controls}")
                print(f"Number of colors: {len(colors)}")

            else:
                print("Unknown colormap type")

    except Exception as e:
        print(f"Error fetching/visualizing colormaps:\n{e}")


# Use the selected object from the interactive picker and fetch colormaps
object_id = selected_object_dropdown.value
await fetch_and_visualize_colormaps(object_id, org_id, workspace_id, connector)